In [64]:
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
import json
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta
import datetime
import re


# ブドウシミュレーター
def grape_calculator_myfive(game, bb, rb, medals, cherry=True):
    bb_medals = 239.25
    rb_medals = 95.25
    replay_rate = 0.411
    if cherry:
        cherry_rate_high = 0.04228
    else:
        cherry_rate_high = 0.05847
    denominator_inner = (
        -medals
        - (
            game * 3
            - (
                bb * bb_medals
                + rb * rb_medals
                + game * replay_rate
                + game * cherry_rate_high
            )
        )
    ) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)

    return grape_rate


def assign_area(unit_no, json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        area_map = json.load(f)
    for rule in area_map:
        if rule["start"] <= unit_no <= rule["end"]:
            return rule["area"]
    return "その他"


def df_preprocessing(df, json_path):
    print(f"データ前処理を行います")
    df_pre = df.copy()
    df_pre["date"] = pd.to_datetime(df_pre["date"])
    df_pre.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
    df_pre = df_pre[
        ["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]
    ]
    df_pre["BB_rate"] = (df_pre["game"] / df_pre["BB"]).round(1)
    df_pre["RB_rate"] = (df_pre["game"] / df_pre["RB"]).round(1)
    df_pre["Grape_rate"] = grape_calculator_myfive(
        df_pre["game"], df_pre["BB"], df_pre["RB"], df_pre["medals"], cherry=True
    ).round(2)
    df_pre["Total_rate"] = (df_pre["game"] / (df_pre["BB"] + df_pre["RB"])).round(1)
    df_pre["month"] = df_pre["date"].dt.strftime("%Y-%m")
    df_pre["day"] = df_pre["date"].dt.day
    df_pre["weekday"] = df_pre["date"].dt.weekday
    df_pre["year"] = df_pre["date"].dt.year
    df_pre["unit_last"] = df_pre["unit_no"].astype(str).str[-1]

    df_pre["area"] = df_pre["unit_no"].apply(lambda x: assign_area(x, json_path))

    df_pre = df_pre.replace([np.inf, -np.inf], np.nan)
    df_pre = df_pre.fillna(0)

    model_list = list(df["model_name"].unique())
    print(f"以下のモデルが含まれています")
    for i, model in enumerate(model_list):
        print(f"{i}: {model}", end=", ")
    return df_pre


def create_df_from_database(HALL_NAME, start_date, end_date):
    # Table name 取得
    DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
    tables = cursor.fetchall()
    # print(tables)

    cursor.execute(
        "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + HALL_NAME + "%",)
    )
    results = cursor.fetchall()

    # 結果表示
    if results:
        for hall_id, hall_name in results:
            print(f"🔍 '{HALL_NAME}' を含むホール名が見つかりました。")
    else:
        print(f"❌ '{HALL_NAME}' を含むホール名は見つかりませんでした。")

    query = """
        -- 出玉データにホール名と機種名を結合して取得
        SELECT
            r.*, 
            h.name AS hall_name,     -- ホール名を追加
            m.name AS model_name     -- 機種名を追加
        FROM results r
        JOIN halls h ON r.hall_id = h.hall_id  -- ホールと結合
        JOIN models m ON r.model_id = m.model_id  -- 機種と結合
        WHERE h.name = ?  -- 指定ホールのみ
        AND m.name LIKE '%ジャグラー%'  -- ジャグラー系機種に限定
        AND r.date BETWEEN ? AND ?  -- 日付範囲を指定
        ORDER BY r.date DESC, r.unit_no ASC;
        """

    df = pd.read_sql_query(query, conn, params=(hall_name, start_date, end_date))
    conn.close()
    print(f"データサイズ: {df.shape[0]} x {df.shape[1]}")
    print(f"📅 対象期間: {start_date} ～ {end_date}")

    return df


def create_pivot_table_date(
    df, start_date, end_date, day_targets, pivot_targets, index_targets, 
    columns_targets
):
    """日付ごとにシートを作成して台ごとの出玉率一覧を作成"""
    df_filtered = df.copy()
    df_filtered = df_filtered[
        (df_filtered["date"].dt.date <= start_date)
        & (df_filtered["date"].dt.date >= end_date)
    ]
    df_filtered = df_filtered[df_filtered["day"] == day_targets]

    pivot_results = {}
    for col in pivot_targets:
        table = df_filtered.pivot_table(
            index=index_targets,
            columns=columns_targets,
            values=col,
            aggfunc="sum",
            margins=True,
            margins_name="Total",
        )
        pivot_results[col] = table.iloc[:, ::-1]

    game = pivot_results["game"]
    medals = pivot_results["medals"]
    rb = pivot_results["RB"]
    bb = pivot_results["BB"]
    rb_rate = (game / rb).round(1)
    total_rate = (game / (bb + rb)).round(1)
    medal_rate = ((medals + game * 3) / (game * 3)).round(3)

    labeled_tables = [
        ("GAME", game),
        ("MEDALS", medals),
        ("RB_RATE", rb_rate),
        ("TOTAL_RATE", total_rate),
        ("MEDAL_RATE", medal_rate),
        ("BB", bb),
        ("RB", rb),
    ]

    # ラベルを MultiIndex に付ける
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合・NaN除去
    interleaved_cols = [
        col
        for pair in zip(
            game.columns,
            medals.columns,
            bb.columns,
            rb.columns,
            medal_rate.columns,
            rb_rate.columns,
            total_rate.columns,
        )
        for col in pair
    ]

    merged = pd.concat([game, medals, medal_rate, bb, rb, rb_rate, total_rate], axis=1)[
        interleaved_cols
    ]
    merged.replace([np.inf, -np.inf, np.nan], None, inplace=True)

    return merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate


# スプレッドシート認証設定
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)

In [65]:

HALL_NAME = "EXA FIRST"
HALL_NAME = "コンサートホールエフ成増"
HALL_NAME = "第一プラザみずほ台店"
# HALL_NAME = "第一プラザ坂戸1000"


SPREADSHEET_IDS = {
    "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
    "コンサートホールエフ成増": "1EDY2RfjDQNsapVrl2X-UrqPKoXrkQmYJnk3uPqccBxY",
    "第一プラザ坂戸1000": "170MVr-BB3LG-g5ItkDT-8TE6R68RW9zJhRfpvQiy-PE",
    "第一プラザみずほ台店": "1_1722pigi_Z1D6eH0tsPfMneGoS9O09fyqD6F-h1mQA",
    # "パールショップともえ川越店": "1i70joJ27Hs7inS-D89z9YMSJO1aRvaBeeWn0n9xpktY",
    # "パラッツォ川越店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
    # "第一プラザ狭山店": "1IVb2Woq3n_PDZP87LdW9NpFP3Z6LeyQtONCkx_fWIq4",
}
spreadsheet = client.open_by_key(SPREADSHEET_IDS[HALL_NAME])

today = datetime.date.today()
start_date = today - relativedelta(months=6, days=today.day - 1)

df = create_df_from_database(
    HALL_NAME, start_date.strftime("%Y-%m-%d"), today.strftime("%Y-%m-%d")
)
JSON_FILE_PATH = f"C:/python/dataOnline/anaslo_02/json/{HALL_NAME}_area_map.json"
df = df_preprocessing(df, JSON_FILE_PATH)
df.head(3)

🔍 '第一プラザみずほ台店' を含むホール名が見つかりました。
データサイズ: 9809 x 11
📅 対象期間: 2024-11-01 ～ 2025-05-19
データ前処理を行います
以下のモデルが含まれています
0: マイジャグラーV, 1: ジャグラーガールズ, 2: アイムジャグラーEX-TP, 3: ファンキージャグラー2, 4: ゴーゴージャグラー3, 5: ハッピージャグラーVIII, 6: ウルトラミラクルジャグラー, 

,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,Grape_rate,Total_rate,month,day,weekday,year,unit_last,area
0,第一プラザみずほ台店,2025-05-18,マイジャグラーV,408,4395,21,8,366,209.3,549.4,6.09,151.6,2025-05,18,6,2025,8,その他
1,第一プラザみずほ台店,2025-05-18,マイジャグラーV,409,743,2,0,-378,371.5,0.0,5.74,371.5,2025-05,18,6,2025,9,その他
2,第一プラザみずほ台店,2025-05-18,マイジャグラーV,410,4915,23,9,733,213.7,546.1,5.71,153.6,2025-05,18,6,2025,0,その他


## 出力データ
- 期間指定
    - 機種別の分析
    - 島別の分析
    - 台番号分析
    - 日付分析
    - 月分析
    - 過去n日の差枚比較
    - 全台系分析

## ピボットテーブル作成

In [52]:
def create_pivot_table(
    df,
    start_date,
    end_date,
    pivot_targets,
    index_targets,
    columns_targets,
    date_reverse=False,
):
    df_filtered = df.copy()
    df_filtered = df_filtered[
        (df_filtered["date"].dt.date <= start_date)
        & (df_filtered["date"].dt.date >= end_date)
    ]

    pivot_results = {}
    for col in pivot_targets:
        table = df_filtered.pivot_table(
            index=index_targets,
            columns=columns_targets,
            values=col,
            aggfunc="sum",
            margins=True,
            margins_name="total",
        )
        if date_reverse:
            pivot_results[col] = table.iloc[:, ::-1]
        else:
            pivot_results[col] = table

    game = pivot_results["game"]
    medals = pivot_results["medals"]
    rb = pivot_results["RB"]
    bb = pivot_results["BB"]
    rb_rate = (game / rb).round(1)
    total_rate = (game / (bb + rb)).round(1)
    medal_rate = ((medals + game * 3) / (game * 3)).round(3)

    labeled_tables = [
        ("GAME", game),
        ("MEDALS", medals),
        ("RB_RATE", rb_rate),
        ("TOTAL_RATE", total_rate),
        ("MEDAL_RATE", medal_rate),
        ("BB", bb),
        ("RB", rb),
    ]

    # ラベルを MultiIndex に付ける
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合・NaN除去
    interleaved_cols = [
        col
        for pair in zip(
            game.columns,
            medals.columns,
            bb.columns,
            rb.columns,
            medal_rate.columns,
            rb_rate.columns,
            total_rate.columns,
        )
        for col in pair
    ]

    merged = pd.concat([game, medals, medal_rate, bb, rb, rb_rate, total_rate], axis=1)[
        interleaved_cols
    ]
    # merged.to_csv(csv_path)

    return merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate

## UNIT_RATE

In [67]:
today = datetime.date.today()
start_date = datetime.date.today()
end_date = start_date - relativedelta(months=6, days=start_date.day - 1)
print(f"対象期間: {start_date} から {end_date}")
print(f"対象ホール: {HALL_NAME}")

csv_path = f"{HALL_NAME}_medal_rate.csv"
pivot_targets = ["game", "medals", "BB", "RB"]
index_targets = ["area", "unit_no"]
columns_targets = ["day"]
merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate = create_pivot_table(
    df,
    start_date,
    end_date,
    pivot_targets,
    index_targets,
    columns_targets,
    date_reverse=False,
)

target_rate = 1.05
medal_rate[("MEDAL_RATE", f"count_{target_rate}+")] = (
    medal_rate.iloc[:, :-1] >= target_rate
).sum(axis=1)
countif = (medal_rate.iloc[:-1, :] >= target_rate).sum(axis=0)
medal_rate = pd.concat(
    [medal_rate, pd.DataFrame([countif], index=[(f"count_{target_rate}+", "")])], axis=0
)
medal_rate.head()

# rows, cols = medal_rate.shape
# sheet_name = f"UNIT_RATE"
# try:
#     worksheet = spreadsheet.worksheet(sheet_name)
#     print(f"✅ シート「{sheet_name}」が既に存在します。")
# except gspread.exceptions.WorksheetNotFound:
#     worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=str(rows+5), cols=str(cols+5))
#     print(f"🆕 シート「{sheet_name}」を新規作成しました。")

# sheet = spreadsheet.worksheet(sheet_name)
# sheet.clear()
# set_with_dataframe(sheet, medal_rate, include_index=True)
# sheet.update_cell(1, 1, today.strftime("%Y-%m-%d UPDATED"))
# print(f"✅ シート '{sheet_name}' に DataFrame を書き込みました！")

対象期間: 2025-05-19 から 2024-11-01
対象ホール: 第一プラザみずほ台店


MEDAL_RATE                                                          \
day              1      2      3      4      5      6      7      8      9   
その他 408      1.048  1.028  0.997  1.029  1.011  1.044  0.993  0.952  0.994   
    409      1.029  0.972  1.027  0.975  0.972  1.060  0.996  0.934  0.933   
    410      1.042  0.943  1.026  0.975  1.023  1.047  0.974  0.977  1.004   
    411      0.994  0.939  1.034  1.026  0.996  1.054  1.031  0.960  0.987   
    412      0.981  0.984  0.943  0.992  1.013  1.018  1.013  0.973  1.036   

                ...                                                          \
day         10  ...     24     25     26     27     28     29     30     31   
その他 408  0.962  ...  0.954  1.009  0.957  0.969  0.949  0.917  0.956  1.012   
    409  0.958  ...  1.000  1.065  1.023  1.042  0.918  0.973  0.935  1.066   
    410  0.936  ...  0.999  1.016  0.958  1.002  0.906  0.962  0.978  1.023   
    411  0.982  ...  0.934  0.927  0.921  1.005  0.981  0.927  1.079  0.981   
    412  1.012  ...  1.010  1.008  1.082  1.039  1.020  0.997  1.000  0.995   

                            
day      total count_1.05+  
その他 408  0.997           0  
    409  1.005           5  
    410  1.010           4  
    411  0.999           4  
    412  1.009           1  

[5 rows x 33 columns]

## ISLAND_RATE

In [54]:
pivot_targets = ["game", "medals", "BB", "RB"]
index_targets = ["area"]
colmns_targets = ["day"]
merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate = create_pivot_table(
    df,
    start_date,
    end_date,
    pivot_targets,
    index_targets,
    colmns_targets,
    date_reverse=False,
)
medal_rate.head(3)

rows, cols = medal_rate.shape
sheet_name = f"ISLAND_RATE"
try:
    worksheet = spreadsheet.worksheet(sheet_name)
    print(f"✅ シート「{sheet_name}」が既に存在します。")
except gspread.exceptions.WorksheetNotFound:
    worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=str(rows+5), cols=str(cols+5))
    print(f"🆕 シート「{sheet_name}」を新規作成しました。")

sheet = spreadsheet.worksheet(sheet_name)
sheet.clear()
set_with_dataframe(sheet, medal_rate, include_index=True)
sheet.update_cell(1, 1, today.strftime("%Y-%m-%d UPDATED"))
print(f"✅ シート '{sheet_name}' に DataFrame を書き込みました！")

🆕 シート「ISLAND_RATE」を新規作成しました。
✅ シート 'ISLAND_RATE' に DataFrame を書き込みました！


## MODEL_RATE

In [ ]:
pivot_targets = ["game", "medals", "BB", "RB"]
index_targets = ["model_name"]
colmns_targets = ["day"]
merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate = create_pivot_table(
    df,
    start_date,
    end_date,
    pivot_targets,
    index_targets,
    colmns_targets,
    date_reverse=False,
)

medal_rate.head(3)

rows, cols = medal_rate.shape
sheet_name = f"MODEL_RATE"
try:
    worksheet = spreadsheet.worksheet(sheet_name)
    print(f"✅ シート「{sheet_name}」が既に存在します。")
except gspread.exceptions.WorksheetNotFound:
    worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=str(rows+5), cols=str(cols+5))
    print(f"🆕 シート「{sheet_name}」を新規作成しました。")

sheet = spreadsheet.worksheet(sheet_name)
sheet.clear()
set_with_dataframe(sheet, medal_rate, include_index=True)
sheet.update_cell(1, 1, today.strftime("%Y-%m-%d UPDATED"))
print(f"✅ シート '{sheet_name}' に DataFrame を書き込みました！")

MEDAL_RATE                                                   \
day                    1      2      3      4      5      6      7      8   
model_name                                                                  
アイムジャグラーEX-TP      0.974  0.976  0.988  1.012  0.994  0.974  1.001  0.984   
ウルトラミラクルジャグラー      0.988  0.945  1.056  0.996  0.982  0.953  0.962  1.010   
ゴーゴージャグラー3         0.966  0.997  0.957  1.002  1.016  1.007  0.988  0.999   

                             ...                                            \
day                9     10  ...     23     24     25     26     27     28   
model_name                   ...                                             
アイムジャグラーEX-TP  0.985  0.995  ...  0.995  0.993  0.981  0.970  0.979  0.974   
ウルトラミラクルジャグラー  0.952  0.990  ...  0.985  0.954  1.003  0.980  0.923  1.009   
ゴーゴージャグラー3     0.942  0.980  ...  0.987  1.020  0.996  0.989  0.950  0.975   

                                           
day               29     30     31  total  
model_name                                 
アイムジャグラーEX-TP  0.982  0.975  0.962  0.990  
ウルトラミラクルジャグラー  0.983  1.015  0.988  0.982  
ゴーゴージャグラー3     0.957  0.995  0.980  0.989  

[3 rows x 32 columns]

# 日付、台番号指定
- SHEET_NAME : DAY1，DAY2，… DAY31

In [56]:
pivot_targets = ["game", "medals", "BB", "RB"]
index_targets = ["area", "unit_no"]
columns_targets = ["date"]

today = datetime.date.today()
for day_target in range(today.day-1, today.day+1):
# for day_target in [4, 5, 13, 11, 14, 15, 17, 20, 23, 31]:
    merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate = create_pivot_table_date(
        df, start_date, end_date, day_target, pivot_targets, index_targets, columns_targets)
    rows, cols = medal_rate.shape
    sheet_name = f"DAY{day_target}"
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
        print(f"✅ シート「{sheet_name}」が既に存在します。")
    except gspread.exceptions.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=str(rows+5), cols=str(cols+5))
        print(f"🆕 シート「{sheet_name}」を新規作成しました。")
    sheet = spreadsheet.worksheet(sheet_name)
    sheet.clear()
    set_with_dataframe(sheet, merged, include_index=True)
    sheet.update_cell(1, 1, today.strftime("%Y-%m-%d : UPDATED"))
    print(f"✅ シート '{sheet_name}' に DataFrame を書き込みました！")

merged.head()

🆕 シート「DAY18」を新規作成しました。
✅ シート 'DAY18' に DataFrame を書き込みました！
🆕 シート「DAY19」を新規作成しました。
✅ シート 'DAY19' に DataFrame を書き込みました！


GAME MEDALS    BB    RB MEDAL_RATE RB_RATE TOTAL_RATE  \
date          Total  Total Total Total      Total   Total      Total   
area unit_no                                                           
その他  408      23603   -244    87    68      0.997   347.1      152.3   
     409      10684    400    42    27      1.012   395.7      154.8   
     410      24492   3065   105    71      1.042   345.0      139.2   
     411      17761  -1217    60    59      0.977   301.0      149.3   
     412      24624   2463   103    75      1.033   328.3      138.3   

                            GAME              MEDALS                  BB  ...  \
date         2025-04-19 00:00:00 2025-04-19 00:00:00 2025-04-19 00:00:00  ...   
area unit_no                                                              ...   
その他  408                  7657.0              1500.0                32.0  ...   
     409                  1168.0              -820.0                 1.0  ...   
     410                  7532.0              4291.0                43.0  ...   
     411                  6260.0              1407.0                25.0  ...   
     412                  7922.0               221.0                32.0  ...   

                      MEDAL_RATE             RB_RATE          TOTAL_RATE  \
date         2024-12-19 00:00:00 2024-12-19 00:00:00 2024-12-19 00:00:00   
area unit_no                                                               
その他  408                   0.933               445.8               185.8   
     409                   0.986                None               191.0   
     410                    0.81              1191.0               297.8   
     411                   0.695               438.0               438.0   
     412                   1.058               625.3               156.3   

                            GAME              MEDALS                  BB  \
date         2024-11-19 00:00:00 2024-11-19 00:00:00 2024-11-19 00:00:00   
area unit_no                                                               
その他  408                  4810.0              -122.0                18.0   
     409                  1746.0             -1198.0                 2.0   
     410                  7265.0              -547.0                24.0   
     411                  4863.0              -390.0                20.0   
     412                  8374.0              3326.0                41.0   

                              RB          MEDAL_RATE             RB_RATE  \
date         2024-11-19 00:00:00 2024-11-19 00:00:00 2024-11-19 00:00:00   
area unit_no                                                               
その他  408                    12.0               0.992               400.8   
     409                     4.0               0.771               436.5   
     410                    22.0               0.975               330.2   
     411                     8.0               0.973               607.9   
     412                    36.0               1.132               232.6   

                      TOTAL_RATE  
date         2024-11-19 00:00:00  
area unit_no                      
その他  408                   160.3  
     409                   291.0  
     410                   157.9  
     411                   173.7  
     412                   108.8  

[5 rows x 42 columns]